# Introduction to FastAPI (Student Version)

**Objective**: Understand FastAPI components, routing, async/sync, parameters, and Pydantic validation.

**Note to Students**: Follow the `TP_FastAPI_Guide.md` instructions to complete the TODOs in this notebook.

## Learning Objectives
- FastAPI fundamentals and setup
- Creating routes with different HTTP methods (GET, POST, PUT, DELETE)
- Path parameters vs Query parameters
- Request/Response models with Pydantic
- Async capabilities
- Automatic API documentation (Swagger)
- REST API principles in practice

## 0. Setting Up FastAPI

First, let's install FastAPI and its dependencies. FastAPI uses Uvicorn as the ASGI server.

In [1]:
# Import required libraries
from fastapi import FastAPI, Query, Path, HTTPException
from pydantic import BaseModel
import uvicorn
from typing import Optional
import asyncio
import threading

## 1. Creating Your First FastAPI App

FastAPI is a modern, fast web framework for building APIs. Let's create a simple app:

In [2]:
# Create a FastAPI instance
app = FastAPI(
    title="FastAPI Learning API",
    description="A simple API to learn FastAPI concepts",
    version="1.0.0"
)

# Basic root endpoint
@app.get("/")
def read_root():
    """Welcome endpoint - returns a simple greeting"""
    # TODO: Return a simple dictionary, e.g., {"message": "Welcome to FastAPI!"}
    return {} 

def run():
    uvicorn.run(app, host="127.0.0.1", port=8000)

threading.Thread(target=run).start()

print("FastAPI server started at http://127.0.0.1:8000")

FastAPI server started at http://127.0.0.1:8000


## 2. Understanding HTTP Methods : GET


In [3]:
# Simple in-memory database for demonstration
items_db = {
    1: {"name": "Laptop", "price": 999.99, "description": "Gaming laptop"},
    2: {"name": "Mouse", "price": 29.99, "description": "Wireless mouse"}
}

# GET - Retrieve data
@app.get("/items")
def get_items():
    """Get all items"""
    return {"items": items_db, "count": len(items_db)}

@app.get("/items/{item_id}")
def get_item(item_id: int):
    """Get a specific item by ID"""
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="Item not found")
    
    return {"item": items_db[item_id], "id": item_id}


## 3. Pydantic Models for Request/Response

Pydantic provides data validation and type hints. Let's create models:

In [ ]:
# Pydantic models for type validation and documentation
class Item(BaseModel):
    name: str
    price: float
    description: Optional[str] = None
    category: Optional[str] = None
    
class ItemResponse(BaseModel):
    id: int
    name: str
    price: float
    description: Optional[str] = None
    
class ItemUpdate(BaseModel):
    name: Optional[str] = None
    price: Optional[float] = None
    description: Optional[str] = None

## 4. Understanding HTTP Methods : POST

In [ ]:
# POST - Create new data
@app.post("/items", response_model=ItemResponse)
def create_item(item: Item):
    """Create a new item"""
    new_id = max(items_db.keys()) + 1
    items_db[new_id] = item.model_dump()
    return ItemResponse(id=new_id, **item.model_dump())

## 5. Understanding HTTP Methods : PUT

In [ ]:
# PUT - Update existing data
@app.put("/items/{item_id}", response_model=ItemResponse)
def update_item(item_id: int, item_update: ItemUpdate):
    """Update an existing item"""
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="Item not found")
    
    stored_item = items_db[item_id]
    update_data = item_update.model_dump(exclude_unset=True)  # Only get fields that were set
    
    for field, value in update_data.items():
        stored_item[field] = value
    
    return ItemResponse(id=item_id, **stored_item)

## 6. Understanding HTTP Methods : DELETE

In [7]:
# DELETE - Remove data
@app.delete("/items/{item_id}")
def delete_item(item_id: int):
    """Delete an item"""
    if item_id in items_db:
        del items_db[item_id]
        return {"message": f"Item {item_id} deleted successfully."}
    pass

## 7. Path Parameters vs Query Parameters

Understanding how to pass data to your API endpoints:

In [8]:
# Path Parameters - part of the URL path
@app.get("/users/{user_id}/items/{item_id}")
def get_user_item(
    user_id: int = Path(..., description="ID of the user", ge=1),
    item_id: int = Path(..., description="ID of the item", ge=1)
):
    """Get an item for a specific user (using path parameters)"""
    return {
        "user_id": user_id,
        "item_id": item_id,
        "message": f"Item {item_id} for user {user_id}"
    }

# Query Parameters - after ? in the URL
@app.get("/search")
def search_items(
    q: Optional[str] = Query(None, description="Search query"),
    min_price: Optional[float] = Query(None, description="Minimum price", ge=0),
    max_price: Optional[float] = Query(None, description="Maximum price", ge=0),
    limit: int = Query(10, description="Number of results", le=100)
):
    """Search items with query parameters"""
    
    # Filter items based on query parameters
    filtered_items = items_db.copy()
    
    if q:
        
        filtered_items = {
            key: value for key, value in filtered_items.items() 
            if q.lower() in value["name"].lower() or (value.get("description") and q.lower() in value["description"].lower())
        }
        
    if min_price is not None:
        filtered_items = {key: value for key, value in filtered_items.items() if value["price"] >= min_price}
    
    # Limit results
    limited_items = dict(list(filtered_items.items())[:limit])
    
    return {
        "query": q,
        "filters": {"min_price": min_price, "max_price": max_price},
        "limit": limit,
        "results": limited_items,
        "total_found": len(limited_items)
    }

## 8. Async vs Synchronous (ASGI vs WSGI)

FastAPI supports both sync and async operations. Here's the difference:

In [9]:
# Synchronous endpoint (blocking)
@app.get("/sync-task")
def sync_heavy_task():
    """Synchronous task that blocks the thread"""
    import time
    time.sleep(2)  # Simulates heavy computation
    return {"message": "Sync task completed", "duration": "2 seconds"}

# Asynchronous endpoint (non-blocking)
@app.get("/async-task")
async def async_heavy_task():
    """Asynchronous task that doesn't block the thread"""
    await asyncio.sleep(2)  # Simulates async I/O operation
    return {"message": "Async task completed", "duration": "2 seconds"}


In [ ]:
@app.get("/health")
def health_check():
    return {"status": "ok"}

INFO:     Started server process [71905]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60352 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:60353 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60353 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60353 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60353 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:60353 - "GET /health HTTP/1.1" 200 OK


## REST API Principles Summary

Let's review the key REST principles we've implemented:

### REST Principles:
1. **Resources**: Everything is a resource (items, users)
2. **HTTP Methods**: Use appropriate methods for actions
   - GET: Retrieve data
   - POST: Create new resources
   - PUT: Update existing resources
   - DELETE: Remove resources
3. **Stateless**: Each request contains all needed information
4. **Uniform Interface**: Consistent URL patterns and response formats
5. **JSON Format**: Standard data exchange format

### FastAPI Key Features:
1. **Automatic Documentation**: Swagger UI and ReDoc
2. **Type Hints**: Python type annotations for validation
3. **Pydantic Models**: Data validation and serialization
4. **Async Support**: High performance with async/await
5. **Path & Query Parameters**: Flexible data passing
6. **Error Handling**: HTTP exceptions for error responses